<a href="https://colab.research.google.com/github/Hannah1123/attentio/blob/master/IntroToTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch.nn as nn
import torch

In [0]:
dummy_point_cloud = torch.rand(2, 900, 3)
dummy_object_classes = torch.randint(high=40, size=(2,))

In [0]:
print(dummy_object_classes)

tensor([13, 22])


Deep Neural network based representation learning of 3D point clouds for stereo scene understanding.

In [0]:
MLP_OUT_DIM = 128
CLASS_NUM = 40
class MyPointNet(nn.Module):
    def __init__(self):
        super(MyPointNet, self).__init__()
        self.mlp = \
            nn.Sequential(
                nn.Linear(in_features=3,
                          out_features=64),
                nn.ReLU(),
            nn.Linear(in_features=64,
                      out_features=MLP_OUT_DIM),
        )
        self.classifier_head = nn.Sequential(
            nn.Linear(in_features=MLP_OUT_DIM, 
                      out_features=MLP_OUT_DIM),
            nn.ReLU(),
            nn.Linear(in_features=MLP_OUT_DIM, 
                      out_features=CLASS_NUM)
        ) # sequential
        
        
        
    def forward(self, pc):
        """
        :param pc: an array of m x n x 3
        """
        # order PC?
        # how let's pretend it were ordered.
        sample_num, point_num, coord_dim = pc.shape
        assert coord_dim == 3, "each point can have 3 features (coordinates in 3D)"
        features = self.mlp(pc.view(-1, 3))\
            .reshape(sample_num, point_num, MLP_OUT_DIM)
        # aggregate the features:
        pc_feat = torch.max(features, dim=1)[0]
        #max的第0维是数据 第一维是该数据所在的位置
        # classific
        out = self.classifier_head(pc_feat)
        return out


In [0]:
pcnet = MyPointNet()

In [0]:
dummy_out = pcnet(dummy)

In [0]:
# dummy_out[0]

# now we interpret the values in output of each point cloud to be the
# log-likelihood for that point cloud belongs to every object class.
# e.g. Prob(PointCloud[0] is of class-3) == exp(dummy_out[0][3])
#                                         ---------------------------
#                                          sum_k exp(dummy_out[0][k])
# In Pytorch, you can use torch.softmax.

dummy_predicted_probability = torch.softmax(dummy_out, dim=1)

In [0]:
dummy_predicted_probability[0], dummy_object_classes[0]

(tensor([0.0250, 0.0265, 0.0262, 0.0282, 0.0275, 0.0238, 0.0281, 0.0219, 0.0236,
         0.0271, 0.0254, 0.0234, 0.0262, 0.0248, 0.0239, 0.0230, 0.0254, 0.0255,
         0.0247, 0.0255, 0.0214, 0.0268, 0.0259, 0.0228, 0.0227, 0.0263, 0.0218,
         0.0277, 0.0229, 0.0262, 0.0255, 0.0250, 0.0263, 0.0220, 0.0260, 0.0246,
         0.0228, 0.0283, 0.0229, 0.0262], grad_fn=<SelectBackward>),
 tensor(13))

In [0]:
A = dummy_predicted_probability[0][dummy_object_classes[0]] + \
    dummy_predicted_probability[1][dummy_object_classes[1]]

In [0]:
A.backward()

In [0]:
params = list(pcnet.parameters())

In [0]:
# params[0].grad
params[0].data += params[0].grad * 0.1

In [0]:
dummy_out2 = pcnet(dummy)

In [0]:
dummy_predicted_probability = torch.softmax(dummy_out2, dim=1)

In [0]:
A2 = dummy_predicted_probability[0][dummy_object_classes[0]] + \
    dummy_predicted_probability[1][dummy_object_classes[1]]

In [0]:
# you don't need to explicitly take softmax (including exp-op)
# you don't need to explicitly take the predicted probability using the class label
# you don't need to manually accumate predicted class prbability
# you don't need to manually maintain the value of the parameters according to
#   the gradient computed



from torch.optim import Adam
optim = Adam(pcnet.parameters(), lr=0.01)

iternum = 0
while True:
    dummy_out = pcnet(dummy)
    loss = torch.nn.functional.nll_loss(dummy_out, dummy_object_classes)
    optim.zero_grad()
    loss.backward()
    optim.step()
    print(loss)
    iternum += 1
    if iternum > 3:
        break



tensor(-0.0266, grad_fn=<NllLossBackward>)
tensor(-0.5715, grad_fn=<NllLossBackward>)
tensor(-1.5681, grad_fn=<NllLossBackward>)
tensor(-3.4721, grad_fn=<NllLossBackward>)


In [0]:
torch.softmax(dummy_out, dim=1)

tensor([[0.0071, 0.0218, 0.0057, 0.0093, 0.0137, 0.0057, 0.0125, 0.0030, 0.0140,
         0.0055, 0.0111, 0.0110, 0.0066, 0.1732, 0.0079, 0.0020, 0.0074, 0.0045,
         0.0081, 0.0204, 0.0048, 0.0149, 0.4436, 0.0039, 0.0117, 0.0071, 0.0093,
         0.0177, 0.0139, 0.0141, 0.0088, 0.0104, 0.0105, 0.0066, 0.0140, 0.0124,
         0.0072, 0.0144, 0.0143, 0.0099],
        [0.0070, 0.0217, 0.0056, 0.0092, 0.0137, 0.0057, 0.0124, 0.0030, 0.0139,
         0.0054, 0.0110, 0.0109, 0.0065, 0.1724, 0.0079, 0.0020, 0.0074, 0.0044,
         0.0080, 0.0202, 0.0048, 0.0149, 0.4472, 0.0039, 0.0116, 0.0071, 0.0092,
         0.0175, 0.0138, 0.0140, 0.0087, 0.0104, 0.0104, 0.0066, 0.0139, 0.0124,
         0.0072, 0.0142, 0.0142, 0.0098]], grad_fn=<SoftmaxBackward>)

In [0]:
0.875 ** 5

0.512908935546875